In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ast
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import normalize
from keras.preprocessing import image
from time import time
from scipy.spatial.distance import cosine, euclidean, hamming

In [ ]:
recipe = pd.read_csv('/kaggle/input/foodrecsysv1/raw-data_recipe.csv')
recipe = recipe.drop(columns=['cooking_directions', 'reviews'])
recipe.head()

In [ ]:
# round average rating into 2 decimal places
def avg_rate(col):
    return f'{col:.2f}'
recipe.aver_rate = recipe.aver_rate.apply(avg_rate)
recipe.head()

In [ ]:
recipe.aver_rate = recipe.aver_rate.astype(float)
recipe.dtypes
list_of_dict = []

for row in recipe.nutritions:
    list_of_dict.append(ast.literal_eval(row))
list_of_dict[1]


'18'
# extract percent daily values for selected nutritions
calories_list = []
fat_list = []
carbohydrates_list = []
protein_list = []
cholesterol_list = []
sodium_list = []
fiber_list = []

for x in range(len(list_of_dict)):
    calories_list.append(list_of_dict[x]['calories']['percentDailyValue'])
    fat_list.append(list_of_dict[x]['fat']['percentDailyValue'])
    carbohydrates_list.append(list_of_dict[x]['carbohydrates']['percentDailyValue'])
    protein_list.append(list_of_dict[x]['protein']['percentDailyValue'])
    cholesterol_list.append(list_of_dict[x]['cholesterol']['percentDailyValue'])
    sodium_list.append(list_of_dict[x]['sodium']['percentDailyValue'])
    fiber_list.append(list_of_dict[x]['fiber']['percentDailyValue'])
    
    
# group all the data into dataframe
data = {'calories': calories_list, 'fat': fat_list, 'carbohydrates': carbohydrates_list, 
       'protein': protein_list, 'cholesterol': cholesterol_list, 'sodium': sodium_list, 
       'fiber': fiber_list}

df = pd.DataFrame(data)
df.index = recipe['recipe_id']
df.head()

In [ ]:
print(df.shape)
df = df.dropna()
def text_cleaning(cols):
    if cols == '< 1':
        return 1
    else:
        return cols

for col in df.columns:
    df[col] = df[col].apply(text_cleaning)
df = df.apply(pd.to_numeric)
# normalized nutrition data by columns
df_normalized = pd.DataFrame(normalize(df, axis=0))
df_normalized.columns = df.columns
df_normalized.index = df.index
df_normalized.head()


# show recipe id, recipe name and image of selected recipe
def selected_recipe(recipe_id):
    image_path = "/kaggle/input/foodrecsysv1/raw-data-images/raw-data-images/{}.jpg"
    image_path = image_path.format(recipe_id)
    
    img = image.load_img(image_path)
    img = image.img_to_array(img, dtype='int')
        
    fig, ax = plt.subplots(1,1)
    ax.imshow(img)
    ax.axis('off')
    
    recipe_df = recipe.set_index('recipe_id')
    x = "{}  {}".format(recipe_id, recipe_df.at[recipe_id, 'recipe_name'])
    ax.set_title(x)
    
    print(df.loc[df.index.isin([recipe_id])].transpose())
    
selected_recipe(87211)


recipe_final= pd.merge(recipe, df, on='recipe_id')
recipe_final= recipe_final.drop(columns=['nutritions'])

recipe_final.to_csv("recipe_final.csv")

In [ ]:
ingredients_list = []

for x in range(len(recipe_final)):
    ingredients_list.append(recipe_final['ingredients'][x].split('^'))
data = {'ingredients_list': ingredients_list}
df1 = pd.DataFrame(data)
df1.index = recipe_final['recipe_id']
df1.head()

recipe_final= pd.merge(recipe_final, df1, on='recipe_id')
recipe_final= recipe_final.drop(columns=['ingredients'])
recipe_final.head()

recipe_final.to_csv("recipe_final.csv")